# Assignment 7
## Task 1 : Read the following data set:
## https://archive.ics.uci.edu/ml/machine-learning-databases/adult/
### Create a sql db from adult dataset and name it sqladb 

In [2]:
import pandas as pd
import sqlite3 as db
from pandasql import sqldf

In [3]:
adultdf=pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",header=None)
adultdf.to_csv("./adult.data.csv",index=None) #Saved file for future use if URL is not available
#adultdf=pd.read_csv("./adult.data.csv")
adultdf.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [59]:
adultdf.shape

(32561, 15)

In [4]:
adultdf.columns = ['age','workclass','fnlwgt','education','education-num',
                   'marital-status','occupation','relationship','race','sex',
                   'capital-gain','capital-loss','hours-per-week','native-country','prediction']
adultdf.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,prediction
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [57]:
cols=['workclass','education','marital-status','occupation','relationship','race','sex','native-country','prediction']
for i in cols:
    adultdf[i]=adultdf[i].str.strip()

In [171]:
adultdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
age               32561 non-null int64
workclass         32561 non-null object
fnlwgt            32561 non-null int64
education         32561 non-null object
education-num     32561 non-null int64
marital-status    32561 non-null object
occupation        32561 non-null object
relationship      32561 non-null object
race              32561 non-null object
sex               32561 non-null object
capital-gain      32561 non-null int64
capital-loss      32561 non-null int64
hours-per-week    32561 non-null int64
native-country    32561 non-null object
prediction        32561 non-null object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [58]:
pysqldf = lambda q: sqldf(q, globals())
pysqldf("SELECT * from adultdf LIMIT 5;")

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,prediction
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [60]:
conn = db.connect('sqladb.db')
cursor = conn.cursor()
sql="""
        CREATE TABLE IF NOT EXISTS ADULT (
        age INTEGER, workclass TEXT, fnlwgt INTEGER, education TEXT, 'education-num' INTEGER, 'marital-status' TEXT, occupation TEXT,
        relationship TEXT,race TEXT, sex TEXT, 'capital-gain' INTEGER, 'capital-loss' INTEGER, 'hours-per-week' INTEGER,
        'native-country' TEXT, prediction TEXT);
        """
cursor.execute(sql)
conn.commit()

In [61]:
cursor.executemany('INSERT INTO ADULT VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)',
                 zip(adultdf.iloc[:,0], adultdf.iloc[:,1], adultdf.iloc[:,2], adultdf.iloc[:,3], adultdf.iloc[:,4],
                     adultdf.iloc[:,5], adultdf.iloc[:,6], adultdf.iloc[:,7], adultdf.iloc[:,8], adultdf.iloc[:,9],
                     adultdf.iloc[:,10], adultdf.iloc[:,11], adultdf.iloc[:,12], adultdf.iloc[:,13], adultdf.iloc[:,14])
                  )
conn.commit()

### 1.1 : Select 10 records from the adult sqladb

In [168]:
cursor.execute('SELECT * FROM ADULT LIMIT 10;')
cursor.fetchall()

[(39,
  'State-gov',
  77516,
  'Bachelors',
  13,
  'Never-married',
  'Adm-clerical',
  'Not-in-family',
  'White',
  'Male',
  2174,
  0,
  40,
  'United-States',
  '<=50K',
  None),
 (50,
  'Self-emp-not-inc',
  83311,
  'Bachelors',
  13,
  'Married-civ-spouse',
  'Exec-managerial',
  'Husband',
  'White',
  'Male',
  0,
  0,
  13,
  'United-States',
  '<=50K',
  None),
 (38,
  'Private',
  215646,
  'HS-grad',
  9,
  'Divorced',
  'Handlers-cleaners',
  'Not-in-family',
  'White',
  'Male',
  0,
  0,
  40,
  'United-States',
  '<=50K',
  None),
 (53,
  'Private',
  234721,
  '11th',
  7,
  'Married-civ-spouse',
  'Handlers-cleaners',
  'Husband',
  'Black',
  'Male',
  0,
  0,
  40,
  'United-States',
  '<=50K',
  None),
 (28,
  'Private',
  338409,
  'Bachelors',
  13,
  'Married-civ-spouse',
  'Prof-specialty',
  'Wife',
  'Black',
  'Female',
  0,
  0,
  40,
  'Cuba',
  '<=50K',
  None),
 (37,
  'Private',
  284582,
  'Masters',
  14,
  'Married-civ-spouse',
  'Exec-managerial

### 1.2 :  Show me the average hours per week of all men who are working in private sector 

In [78]:
cursor.execute("SELECT AVG(\"hours-per-week\") FROM ADULT where workclass = 'Private' and sex = 'Male';")
cursor.fetchall()

[(42.22122591006424,)]

 ### 1.3 : Show me the frequency table for education, occupation and relationship, separately

In [79]:
cursor.execute("SELECT count(education),education FROM ADULT group by education;")
cursor.fetchall()

[(933, '10th'),
 (1175, '11th'),
 (433, '12th'),
 (168, '1st-4th'),
 (333, '5th-6th'),
 (646, '7th-8th'),
 (514, '9th'),
 (1067, 'Assoc-acdm'),
 (1382, 'Assoc-voc'),
 (5355, 'Bachelors'),
 (413, 'Doctorate'),
 (10501, 'HS-grad'),
 (1723, 'Masters'),
 (51, 'Preschool'),
 (576, 'Prof-school'),
 (7291, 'Some-college')]

In [80]:
cursor.execute("SELECT count(occupation),occupation FROM ADULT group by occupation;")
cursor.fetchall()

[(1843, '?'),
 (3770, 'Adm-clerical'),
 (9, 'Armed-Forces'),
 (4099, 'Craft-repair'),
 (4066, 'Exec-managerial'),
 (994, 'Farming-fishing'),
 (1370, 'Handlers-cleaners'),
 (2002, 'Machine-op-inspct'),
 (3295, 'Other-service'),
 (149, 'Priv-house-serv'),
 (4140, 'Prof-specialty'),
 (649, 'Protective-serv'),
 (3650, 'Sales'),
 (928, 'Tech-support'),
 (1597, 'Transport-moving')]

In [81]:
cursor.execute("SELECT count(relationship),relationship FROM ADULT group by relationship;")
cursor.fetchall()

[(13193, 'Husband'),
 (8305, 'Not-in-family'),
 (981, 'Other-relative'),
 (5068, 'Own-child'),
 (3446, 'Unmarried'),
 (1568, 'Wife')]

### 1.4 : Are there any people who are married, working in private sector and having a masters degree 

In [82]:
cursor.execute("SELECT count(1) FROM ADULT where relationship in ('Husband','Wife') and workclass = 'Private' and education='Masters';")
cursor.fetchall()

[(528,)]

### 1.5 :  What is the average, minimum and maximum age group for people working in different sectors 

In [154]:
cursor.execute("SELECT avg(age),min(age),max(age),workclass FROM ADULT group by workclass ;")
cursor.fetchall()

[(40.960239651416124, 17, 90, '?'),
 (42.590625, 17, 90, 'Federal-gov'),
 (41.75107501194458, 17, 90, 'Local-gov'),
 (20.571428571428573, 17, 30, 'Never-worked'),
 (36.797585477617204, 17, 90, 'Private'),
 (46.01702508960574, 17, 84, 'Self-emp-inc'),
 (44.96969696969697, 17, 90, 'Self-emp-not-inc'),
 (39.43605546995377, 17, 81, 'State-gov'),
 (47.785714285714285, 19, 72, 'Without-pay')]

### 1.6 : Calculate age distribution by country

In [156]:
cursor.execute("SELECT count(age),age,\"native-country\" FROM ADULT group by age,\"native-country\";")
cursor.fetchall()

[(2, 17, '?'),
 (2, 17, 'Canada'),
 (2, 17, 'El-Salvador'),
 (1, 17, 'England'),
 (1, 17, 'Haiti'),
 (1, 17, 'India'),
 (6, 17, 'Mexico'),
 (2, 17, 'Peru'),
 (1, 17, 'Philippines'),
 (1, 17, 'Poland'),
 (1, 17, 'Puerto-Rico'),
 (1, 17, 'Trinadad&Tobago'),
 (374, 17, 'United-States'),
 (8, 18, '?'),
 (1, 18, 'Cambodia'),
 (1, 18, 'Canada'),
 (1, 18, 'Columbia'),
 (1, 18, 'Dominican-Republic'),
 (1, 18, 'Germany'),
 (1, 18, 'Honduras'),
 (1, 18, 'India'),
 (1, 18, 'Jamaica'),
 (6, 18, 'Mexico'),
 (1, 18, 'Peru'),
 (2, 18, 'Philippines'),
 (1, 18, 'Scotland'),
 (524, 18, 'United-States'),
 (5, 19, '?'),
 (1, 19, 'Canada'),
 (4, 19, 'El-Salvador'),
 (2, 19, 'England'),
 (3, 19, 'Guatemala'),
 (1, 19, 'Haiti'),
 (1, 19, 'Honduras'),
 (1, 19, 'Hong'),
 (1, 19, 'Italy'),
 (1, 19, 'Jamaica'),
 (2, 19, 'Japan'),
 (2, 19, 'Laos'),
 (14, 19, 'Mexico'),
 (2, 19, 'Nicaragua'),
 (2, 19, 'Philippines'),
 (1, 19, 'Portugal'),
 (3, 19, 'Puerto-Rico'),
 (3, 19, 'South'),
 (2, 19, 'Thailand'),
 (659, 19,

### 1.7 : Compute a new column as 'Net-Capital-Gain' from the two columns 'capital-gain' and 'capital-loss' 

In [167]:
cursor.execute("SELECT (\"capital-gain\" - \"capital-loss\") AS \"Net-Capital-Gain\" FROM ADULT LIMIT 15;")
cursor.fetchall()

[(2174,),
 (0,),
 (0,),
 (0,),
 (0,),
 (0,),
 (0,),
 (0,),
 (14084,),
 (5178,),
 (0,),
 (0,),
 (0,),
 (0,),
 (0,)]

## Task 2 :  SQLALchemy
### Read the following data set: https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data

In [205]:
import pandas as pd
addf=pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",header=None)
addf.columns = ['age','workclass','fnlwgt','education','education_num',
                   'martial_status','occupation','relationship','race','sex',
                   'capital_gain','capital_loss','hours_per_week','native_country','prediction']
addf.head()

,age,workclass,fnlwgt,education,education_num,martial_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,prediction
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


### 2.1 : Create an sqlalchemy engine using a sample from the data set

In [264]:
import sqlalchemy
from sqlalchemy import create_engine,Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

eng = create_engine('sqlite:///sqlalchemy_adult.db', echo=True)
Base = declarative_base()
session_mk = sessionmaker(bind=eng)
session=session_mk()

class Adult(Base) :
    __tablename__ = 'adult'
    age=Column(Integer)
    workclass=Column(String)
    fnlwgt=Column(Integer, primary_key=True)
    education=Column(String)
    education_num=Column(Integer)
    martial_status=Column(String)
    occupation=Column(String)
    relationship=Column(String)
    race=Column(String)
    sex=Column(String)
    capital_gain=Column(Integer)
    capital_loss=Column(Integer)
    hours_per_week=Column(Integer)
    native_country=Column(String)
    prediction=Column(String)
    
    def __repr__(self):
        return "<Adult(age='%d',workclass='%s', fnlwgt='%d', education='%s', education_num='%d', martial_status='%s', occupation=%s, relationship=%s, race=%s, sex='%s', capital_gain='%d', capital_loss='%d', hours_per_week='%d', native_country='%s', prediction='%s')>" % (self.age, self.workclass, self.fnlwgt, self.education, self.education_num, self.martial_status, self.occupation, self.relationship, self.race, self.sex, self.capital_gain, self.capital_loss, self.hours_per_week, self.native_country, self.prediction)
    
Base.metadata.create_all(eng)

addf[0:101].to_sql('adult', con=eng, index=False, if_exists='replace')

2019-03-21 10:52:55,572 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1


INFO:sqlalchemy.engine.base.Engine:SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1


2019-03-21 10:52:55,574 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


2019-03-21 10:52:55,578 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1


...

INFO:sqlalchemy.engine.base.Engine:INSERT INTO adult (age, workclass, fnlwgt, education, education_num, martial_status, occupation, relationship, race, sex, capital_gain, capital_loss, hours_per_week, native_country, prediction) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)


2019-03-21 10:52:56,017 INFO sqlalchemy.engine.base.Engine ((39, ' State-gov', 77516, ' Bachelors', 13, ' Never-married', ' Adm-clerical', ' Not-in-family', ' White', ' Male', 2174, 0, 40, ' United-States', ' <=50K'), (50, ' Self-emp-not-inc', 83311, ' Bachelors', 13, ' Married-civ-spouse', ' Exec-managerial', ' Husband', ' White', ' Male', 0, 0, 13, ' United-States', ' <=50K'), (38, ' Private', 215646, ' HS-grad', 9, ' Divorced', ' Handlers-cleaners', ' Not-in-family', ' White', ' Male', 0, 0, 40, ' United-States', ' <=50K'), (53, ' Private', 234721, ' 11th', 7, ' Married-civ-spouse', ' Handlers-cleaners', ' Husband', ' Black', ' Male', 0, 0, 40, ' United-States', ' <=50K'), (28, ' Private', 338409, ' Bachelors', 13, ' Married-civ-spouse', ' Prof-specialty', ' Wife', ' Black', ' Female', 0, 0, 40, ' Cuba', ' <=50K'), (37, ' Private', 284582, ' Masters', 14, ' Married-civ-spouse', ' Exec-managerial', ' Wife', ' White', ' Female', 0, 0, 40, ' United-States', ' <=50K'), (49, ' Private', 

INFO:sqlalchemy.engine.base.Engine:((39, ' State-gov', 77516, ' Bachelors', 13, ' Never-married', ' Adm-clerical', ' Not-in-family', ' White', ' Male', 2174, 0, 40, ' United-States', ' <=50K'), (50, ' Self-emp-not-inc', 83311, ' Bachelors', 13, ' Married-civ-spouse', ' Exec-managerial', ' Husband', ' White', ' Male', 0, 0, 13, ' United-States', ' <=50K'), (38, ' Private', 215646, ' HS-grad', 9, ' Divorced', ' Handlers-cleaners', ' Not-in-family', ' White', ' Male', 0, 0, 40, ' United-States', ' <=50K'), (53, ' Private', 234721, ' 11th', 7, ' Married-civ-spouse', ' Handlers-cleaners', ' Husband', ' Black', ' Male', 0, 0, 40, ' United-States', ' <=50K'), (28, ' Private', 338409, ' Bachelors', 13, ' Married-civ-spouse', ' Prof-specialty', ' Wife', ' Black', ' Female', 0, 0, 40, ' Cuba', ' <=50K'), (37, ' Private', 284582, ' Masters', 14, ' Married-civ-spouse', ' Exec-managerial', ' Wife', ' White', ' Female', 0, 0, 40, ' United-States', ' <=50K'), (49, ' Private', 160187, ' 9th', 5, ' Mar

2019-03-21 10:52:56,028 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


### 2.2 : Write two basic update queries

In [271]:
adult1 = session.query(Adult).filter_by(age='31').all()
adult1

2019-03-21 10:55:08,508 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.base.Engine:BEGIN (implicit)


2019-03-21 10:55:08,514 INFO sqlalchemy.engine.base.Engine SELECT adult.age AS adult_age, adult.workclass AS adult_workclass, adult.fnlwgt AS adult_fnlwgt, adult.education AS adult_education, adult.education_num AS adult_education_num, adult.martial_status AS adult_martial_status, adult.occupation AS adult_occupation, adult.relationship AS adult_relationship, adult.race AS adult_race, adult.sex AS adult_sex, adult.capital_gain AS adult_capital_gain, adult.capital_loss AS adult_capital_loss, adult.hours_per_week AS adult_hours_per_week, adult.native_country AS adult_native_country, adult.prediction AS adult_prediction 
FROM adult 
WHERE adult.age = ?


INFO:sqlalchemy.engine.base.Engine:SELECT adult.age AS adult_age, adult.workclass AS adult_workclass, adult.fnlwgt AS adult_fnlwgt, adult.education AS adult_education, adult.education_num AS adult_education_num, adult.martial_status AS adult_martial_status, adult.occupation AS adult_occupation, adult.relationship AS adult_relationship, adult.race AS adult_race, adult.sex AS adult_sex, adult.capital_gain AS adult_capital_gain, adult.capital_loss AS adult_capital_loss, adult.hours_per_week AS adult_hours_per_week, adult.native_country AS adult_native_country, adult.prediction AS adult_prediction 
FROM adult 
WHERE adult.age = ?


2019-03-21 10:55:08,517 INFO sqlalchemy.engine.base.Engine ('31',)


INFO:sqlalchemy.engine.base.Engine:('31',)


[<Adult(age='31',workclass=' Private', fnlwgt='45781', education=' Masters', education_num='14', martial_status=' Never-married', occupation= Prof-specialty, relationship= Not-in-family, race= White, sex=' Female', capital_gain='14084', capital_loss='0', hours_per_week='50', native_country=' United-States', prediction=' >50K')>,
 <Adult(age='31',workclass=' Private', fnlwgt='84154', education=' Some-college', education_num='10', martial_status=' Married-civ-spouse', occupation= Sales, relationship= Husband, race= White, sex=' Male', capital_gain='0', capital_loss='0', hours_per_week='38', native_country=' ?', prediction=' >50K')>,
 <Adult(age='31',workclass=' Private', fnlwgt='507875', education=' 9th', education_num='5', martial_status=' Married-civ-spouse', occupation= Machine-op-inspct, relationship= Husband, race= White, sex=' Male', capital_gain='0', capital_loss='0', hours_per_week='43', native_country=' United-States', prediction=' <=50K')>,
 <Adult(age='31',workclass=' Private'

In [272]:
adult2 = session.query(Adult).filter_by(education=' 11th').all()
adult2

2019-03-21 10:55:17,322 INFO sqlalchemy.engine.base.Engine SELECT adult.age AS adult_age, adult.workclass AS adult_workclass, adult.fnlwgt AS adult_fnlwgt, adult.education AS adult_education, adult.education_num AS adult_education_num, adult.martial_status AS adult_martial_status, adult.occupation AS adult_occupation, adult.relationship AS adult_relationship, adult.race AS adult_race, adult.sex AS adult_sex, adult.capital_gain AS adult_capital_gain, adult.capital_loss AS adult_capital_loss, adult.hours_per_week AS adult_hours_per_week, adult.native_country AS adult_native_country, adult.prediction AS adult_prediction 
FROM adult 
WHERE adult.education = ?


INFO:sqlalchemy.engine.base.Engine:SELECT adult.age AS adult_age, adult.workclass AS adult_workclass, adult.fnlwgt AS adult_fnlwgt, adult.education AS adult_education, adult.education_num AS adult_education_num, adult.martial_status AS adult_martial_status, adult.occupation AS adult_occupation, adult.relationship AS adult_relationship, adult.race AS adult_race, adult.sex AS adult_sex, adult.capital_gain AS adult_capital_gain, adult.capital_loss AS adult_capital_loss, adult.hours_per_week AS adult_hours_per_week, adult.native_country AS adult_native_country, adult.prediction AS adult_prediction 
FROM adult 
WHERE adult.education = ?


2019-03-21 10:55:17,326 INFO sqlalchemy.engine.base.Engine (' 11th',)


INFO:sqlalchemy.engine.base.Engine:(' 11th',)


[<Adult(age='53',workclass=' Private', fnlwgt='234721', education=' 11th', education_num='7', martial_status=' Married-civ-spouse', occupation= Handlers-cleaners, relationship= Husband, race= Black, sex=' Male', capital_gain='0', capital_loss='0', hours_per_week='40', native_country=' United-States', prediction=' <=50K')>,
 <Adult(age='38',workclass=' Private', fnlwgt='28887', education=' 11th', education_num='7', martial_status=' Married-civ-spouse', occupation= Sales, relationship= Husband, race= White, sex=' Male', capital_gain='0', capital_loss='0', hours_per_week='50', native_country=' United-States', prediction=' <=50K')>,
 <Adult(age='43',workclass=' Private', fnlwgt='117037', education=' 11th', education_num='7', martial_status=' Married-civ-spouse', occupation= Transport-moving, relationship= Husband, race= White, sex=' Male', capital_gain='0', capital_loss='2042', hours_per_week='40', native_country=' United-States', prediction=' <=50K')>,
 <Adult(age='48',workclass=' Private

In [273]:
adult1[0].education = 'PhD'
adult2[0].sex = 'Female'
session.dirty
session.commit()

2019-03-21 10:55:38,876 INFO sqlalchemy.engine.base.Engine UPDATE adult SET education=? WHERE adult.fnlwgt = ?


INFO:sqlalchemy.engine.base.Engine:UPDATE adult SET education=? WHERE adult.fnlwgt = ?


2019-03-21 10:55:38,878 INFO sqlalchemy.engine.base.Engine ('PhD', 45781)


INFO:sqlalchemy.engine.base.Engine:('PhD', 45781)


2019-03-21 10:55:38,915 INFO sqlalchemy.engine.base.Engine UPDATE adult SET sex=? WHERE adult.fnlwgt = ?


INFO:sqlalchemy.engine.base.Engine:UPDATE adult SET sex=? WHERE adult.fnlwgt = ?


2019-03-21 10:55:38,917 INFO sqlalchemy.engine.base.Engine ('Female', 234721)


INFO:sqlalchemy.engine.base.Engine:('Female', 234721)


2019-03-21 10:55:38,921 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


In [274]:
adult1

2019-03-21 10:55:44,771 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.base.Engine:BEGIN (implicit)


2019-03-21 10:55:44,775 INFO sqlalchemy.engine.base.Engine SELECT adult.age AS adult_age, adult.workclass AS adult_workclass, adult.fnlwgt AS adult_fnlwgt, adult.education AS adult_education, adult.education_num AS adult_education_num, adult.martial_status AS adult_martial_status, adult.occupation AS adult_occupation, adult.relationship AS adult_relationship, adult.race AS adult_race, adult.sex AS adult_sex, adult.capital_gain AS adult_capital_gain, adult.capital_loss AS adult_capital_loss, adult.hours_per_week AS adult_hours_per_week, adult.native_country AS adult_native_country, adult.prediction AS adult_prediction 
FROM adult 
WHERE adult.fnlwgt = ?


INFO:sqlalchemy.engine.base.Engine:SELECT adult.age AS adult_age, adult.workclass AS adult_workclass, adult.fnlwgt AS adult_fnlwgt, adult.education AS adult_education, adult.education_num AS adult_education_num, adult.martial_status AS adult_martial_status, adult.occupation AS adult_occupation, adult.relationship AS adult_relationship, adult.race AS adult_race, adult.sex AS adult_sex, adult.capital_gain AS adult_capital_gain, adult.capital_loss AS adult_capital_loss, adult.hours_per_week AS adult_hours_per_week, adult.native_country AS adult_native_country, adult.prediction AS adult_prediction 
FROM adult 
WHERE adult.fnlwgt = ?


2019-03-21 10:55:44,778 INFO sqlalchemy.engine.base.Engine (45781,)


...

2019-03-21 10:55:44,807 INFO sqlalchemy.engine.base.Engine SELECT adult.age AS adult_age, adult.workclass AS adult_workclass, adult.fnlwgt AS adult_fnlwgt, adult.education AS adult_education, adult.education_num AS adult_education_num, adult.martial_status AS adult_martial_status, adult.occupation AS adult_occupation, adult.relationship AS adult_relationship, adult.race AS adult_race, adult.sex AS adult_sex, adult.capital_gain AS adult_capital_gain, adult.capital_loss AS adult_capital_loss, adult.hours_per_week AS adult_hours_per_week, adult.native_country AS adult_native_country, adult.prediction AS adult_prediction 
FROM adult 
WHERE adult.fnlwgt = ?


INFO:sqlalchemy.engine.base.Engine:SELECT adult.age AS adult_age, adult.workclass AS adult_workclass, adult.fnlwgt AS adult_fnlwgt, adult.education AS adult_education, adult.education_num AS adult_education_num, adult.martial_status AS adult_martial_status, adult.occupation AS adult_occupation, adult.relationship AS adult_relationship, adult.race AS adult_race, adult.sex AS adult_sex, adult.capital_gain AS adult_capital_gain, adult.capital_loss AS adult_capital_loss, adult.hours_per_week AS adult_hours_per_week, adult.native_country AS adult_native_country, adult.prediction AS adult_prediction 
FROM adult 
WHERE adult.fnlwgt = ?


2019-03-21 10:55:44,809 INFO sqlalchemy.engine.base.Engine (125927,)


INFO:sqlalchemy.engine.base.Engine:(125927,)


[<Adult(age='31',workclass=' Private', fnlwgt='45781', education='PhD', education_num='14', martial_status=' Never-married', occupation= Prof-specialty, relationship= Not-in-family, race= White, sex=' Female', capital_gain='14084', capital_loss='0', hours_per_week='50', native_country=' United-States', prediction=' >50K')>,
 <Adult(age='31',workclass=' Private', fnlwgt='84154', education=' Some-college', education_num='10', martial_status=' Married-civ-spouse', occupation= Sales, relationship= Husband, race= White, sex=' Male', capital_gain='0', capital_loss='0', hours_per_week='38', native_country=' ?', prediction=' >50K')>,
 <Adult(age='31',workclass=' Private', fnlwgt='507875', education=' 9th', education_num='5', martial_status=' Married-civ-spouse', occupation= Machine-op-inspct, relationship= Husband, race= White, sex=' Male', capital_gain='0', capital_loss='0', hours_per_week='43', native_country=' United-States', prediction=' <=50K')>,
 <Adult(age='31',workclass=' Private', fnl

In [275]:
adult2

2019-03-21 10:56:00,444 INFO sqlalchemy.engine.base.Engine SELECT adult.age AS adult_age, adult.workclass AS adult_workclass, adult.fnlwgt AS adult_fnlwgt, adult.education AS adult_education, adult.education_num AS adult_education_num, adult.martial_status AS adult_martial_status, adult.occupation AS adult_occupation, adult.relationship AS adult_relationship, adult.race AS adult_race, adult.sex AS adult_sex, adult.capital_gain AS adult_capital_gain, adult.capital_loss AS adult_capital_loss, adult.hours_per_week AS adult_hours_per_week, adult.native_country AS adult_native_country, adult.prediction AS adult_prediction 
FROM adult 
WHERE adult.fnlwgt = ?


INFO:sqlalchemy.engine.base.Engine:SELECT adult.age AS adult_age, adult.workclass AS adult_workclass, adult.fnlwgt AS adult_fnlwgt, adult.education AS adult_education, adult.education_num AS adult_education_num, adult.martial_status AS adult_martial_status, adult.occupation AS adult_occupation, adult.relationship AS adult_relationship, adult.race AS adult_race, adult.sex AS adult_sex, adult.capital_gain AS adult_capital_gain, adult.capital_loss AS adult_capital_loss, adult.hours_per_week AS adult_hours_per_week, adult.native_country AS adult_native_country, adult.prediction AS adult_prediction 
FROM adult 
WHERE adult.fnlwgt = ?


2019-03-21 10:56:00,447 INFO sqlalchemy.engine.base.Engine (234721,)


INFO:sqlalchemy.engine.base.Engine:(234721,)


2019-03-21 10:56:00,453 INFO sqlalchemy.engine.base.Engine SELECT adult.age AS adult_age, adult.workclass AS adult_workclass, adult.fnlwgt AS adult_fnlwgt, adult.education AS adult_education, adult.education_num AS adult_education_num, adult.martial_status AS adult_martial_status, adult.occupation AS adult_occupation, adult.relationship AS adult_relationship, adult.race AS adult_race, adult.sex AS adult_sex, adult.capital_gain AS adult_capital_gain, adult.capital_loss AS adult_capital_loss, adult.hours_per_week AS adult_hours_per_week, adult.native_country AS adult_native_country, adult.prediction AS adult_prediction 
FROM adult 
WHERE adult.fnlwgt = ?


...

2019-03-21 10:56:00,477 INFO sqlalchemy.engine.base.Engine SELECT adult.age AS adult_age, adult.workclass AS adult_workclass, adult.fnlwgt AS adult_fnlwgt, adult.education AS adult_education, adult.education_num AS adult_education_num, adult.martial_status AS adult_martial_status, adult.occupation AS adult_occupation, adult.relationship AS adult_relationship, adult.race AS adult_race, adult.sex AS adult_sex, adult.capital_gain AS adult_capital_gain, adult.capital_loss AS adult_capital_loss, adult.hours_per_week AS adult_hours_per_week, adult.native_country AS adult_native_country, adult.prediction AS adult_prediction 
FROM adult 
WHERE adult.fnlwgt = ?


INFO:sqlalchemy.engine.base.Engine:SELECT adult.age AS adult_age, adult.workclass AS adult_workclass, adult.fnlwgt AS adult_fnlwgt, adult.education AS adult_education, adult.education_num AS adult_education_num, adult.martial_status AS adult_martial_status, adult.occupation AS adult_occupation, adult.relationship AS adult_relationship, adult.race AS adult_race, adult.sex AS adult_sex, adult.capital_gain AS adult_capital_gain, adult.capital_loss AS adult_capital_loss, adult.hours_per_week AS adult_hours_per_week, adult.native_country AS adult_native_country, adult.prediction AS adult_prediction 
FROM adult 
WHERE adult.fnlwgt = ?


2019-03-21 10:56:00,479 INFO sqlalchemy.engine.base.Engine (309634,)


INFO:sqlalchemy.engine.base.Engine:(309634,)


[<Adult(age='53',workclass=' Private', fnlwgt='234721', education=' 11th', education_num='7', martial_status=' Married-civ-spouse', occupation= Handlers-cleaners, relationship= Husband, race= Black, sex='Female', capital_gain='0', capital_loss='0', hours_per_week='40', native_country=' United-States', prediction=' <=50K')>,
 <Adult(age='38',workclass=' Private', fnlwgt='28887', education=' 11th', education_num='7', martial_status=' Married-civ-spouse', occupation= Sales, relationship= Husband, race= White, sex=' Male', capital_gain='0', capital_loss='0', hours_per_week='50', native_country=' United-States', prediction=' <=50K')>,
 <Adult(age='43',workclass=' Private', fnlwgt='117037', education=' 11th', education_num='7', martial_status=' Married-civ-spouse', occupation= Transport-moving, relationship= Husband, race= White, sex=' Male', capital_gain='0', capital_loss='2042', hours_per_week='40', native_country=' United-States', prediction=' <=50K')>,
 <Adult(age='48',workclass=' Privat

In [276]:
from sqlalchemy import update
stmt = update(Adult).where(Adult.workclass==' State-gov').values(hours_per_week='8')
session.execute(stmt)
session.rollback()

2019-03-21 10:56:11,193 INFO sqlalchemy.engine.base.Engine UPDATE adult SET hours_per_week=? WHERE adult.workclass = ?


INFO:sqlalchemy.engine.base.Engine:UPDATE adult SET hours_per_week=? WHERE adult.workclass = ?


2019-03-21 10:56:11,196 INFO sqlalchemy.engine.base.Engine ('8', ' State-gov')


INFO:sqlalchemy.engine.base.Engine:('8', ' State-gov')


2019-03-21 10:56:11,202 INFO sqlalchemy.engine.base.Engine ROLLBACK


INFO:sqlalchemy.engine.base.Engine:ROLLBACK


### 2.3 : Write two delete queries

In [277]:
from sqlalchemy import delete,and_,or_
stmt1 = delete(Adult).where(and_(Adult.workclass==' State-gov',Adult.age=='31',Adult.sex==' Male'))
stmt2 = delete(Adult).where(or_(Adult.capital_gain=='0',Adult.hours_per_week<'30'))
session.execute(stmt1)
session.execute(stmt2)
session.commit()

2019-03-21 10:56:20,234 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.base.Engine:BEGIN (implicit)


2019-03-21 10:56:20,237 INFO sqlalchemy.engine.base.Engine DELETE FROM adult WHERE adult.workclass = ? AND adult.age = ? AND adult.sex = ?


INFO:sqlalchemy.engine.base.Engine:DELETE FROM adult WHERE adult.workclass = ? AND adult.age = ? AND adult.sex = ?


2019-03-21 10:56:20,240 INFO sqlalchemy.engine.base.Engine (' State-gov', '31', ' Male')


...

INFO:sqlalchemy.engine.base.Engine:DELETE FROM adult WHERE adult.capital_gain = ? OR adult.hours_per_week < ?


2019-03-21 10:56:20,245 INFO sqlalchemy.engine.base.Engine ('0', '30')


INFO:sqlalchemy.engine.base.Engine:('0', '30')


2019-03-21 10:56:20,640 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


### 2.4 : Write two filter queries

In [278]:
adult3 = session.query(Adult).filter(Adult.age>='31').all()
adult3

2019-03-21 10:56:28,303 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.base.Engine:BEGIN (implicit)


2019-03-21 10:56:28,307 INFO sqlalchemy.engine.base.Engine SELECT adult.age AS adult_age, adult.workclass AS adult_workclass, adult.fnlwgt AS adult_fnlwgt, adult.education AS adult_education, adult.education_num AS adult_education_num, adult.martial_status AS adult_martial_status, adult.occupation AS adult_occupation, adult.relationship AS adult_relationship, adult.race AS adult_race, adult.sex AS adult_sex, adult.capital_gain AS adult_capital_gain, adult.capital_loss AS adult_capital_loss, adult.hours_per_week AS adult_hours_per_week, adult.native_country AS adult_native_country, adult.prediction AS adult_prediction 
FROM adult 
WHERE adult.age >= ?


INFO:sqlalchemy.engine.base.Engine:SELECT adult.age AS adult_age, adult.workclass AS adult_workclass, adult.fnlwgt AS adult_fnlwgt, adult.education AS adult_education, adult.education_num AS adult_education_num, adult.martial_status AS adult_martial_status, adult.occupation AS adult_occupation, adult.relationship AS adult_relationship, adult.race AS adult_race, adult.sex AS adult_sex, adult.capital_gain AS adult_capital_gain, adult.capital_loss AS adult_capital_loss, adult.hours_per_week AS adult_hours_per_week, adult.native_country AS adult_native_country, adult.prediction AS adult_prediction 
FROM adult 
WHERE adult.age >= ?


2019-03-21 10:56:28,310 INFO sqlalchemy.engine.base.Engine ('31',)


INFO:sqlalchemy.engine.base.Engine:('31',)


[<Adult(age='39',workclass=' State-gov', fnlwgt='77516', education=' Bachelors', education_num='13', martial_status=' Never-married', occupation= Adm-clerical, relationship= Not-in-family, race= White, sex=' Male', capital_gain='2174', capital_loss='0', hours_per_week='40', native_country=' United-States', prediction=' <=50K')>,
 <Adult(age='31',workclass=' Private', fnlwgt='45781', education='PhD', education_num='14', martial_status=' Never-married', occupation= Prof-specialty, relationship= Not-in-family, race= White, sex=' Female', capital_gain='14084', capital_loss='0', hours_per_week='50', native_country=' United-States', prediction=' >50K')>,
 <Adult(age='42',workclass=' Private', fnlwgt='159449', education=' Bachelors', education_num='13', martial_status=' Married-civ-spouse', occupation= Exec-managerial, relationship= Husband, race= White, sex=' Male', capital_gain='5178', capital_loss='0', hours_per_week='40', native_country=' United-States', prediction=' >50K')>,
 <Adult(age=

In [279]:
adult4 = session.query(Adult).filter(Adult.race.like('%te')).all()
adult4

2019-03-21 10:56:36,776 INFO sqlalchemy.engine.base.Engine SELECT adult.age AS adult_age, adult.workclass AS adult_workclass, adult.fnlwgt AS adult_fnlwgt, adult.education AS adult_education, adult.education_num AS adult_education_num, adult.martial_status AS adult_martial_status, adult.occupation AS adult_occupation, adult.relationship AS adult_relationship, adult.race AS adult_race, adult.sex AS adult_sex, adult.capital_gain AS adult_capital_gain, adult.capital_loss AS adult_capital_loss, adult.hours_per_week AS adult_hours_per_week, adult.native_country AS adult_native_country, adult.prediction AS adult_prediction 
FROM adult 
WHERE adult.race LIKE ?


INFO:sqlalchemy.engine.base.Engine:SELECT adult.age AS adult_age, adult.workclass AS adult_workclass, adult.fnlwgt AS adult_fnlwgt, adult.education AS adult_education, adult.education_num AS adult_education_num, adult.martial_status AS adult_martial_status, adult.occupation AS adult_occupation, adult.relationship AS adult_relationship, adult.race AS adult_race, adult.sex AS adult_sex, adult.capital_gain AS adult_capital_gain, adult.capital_loss AS adult_capital_loss, adult.hours_per_week AS adult_hours_per_week, adult.native_country AS adult_native_country, adult.prediction AS adult_prediction 
FROM adult 
WHERE adult.race LIKE ?


2019-03-21 10:56:36,779 INFO sqlalchemy.engine.base.Engine ('%te',)


INFO:sqlalchemy.engine.base.Engine:('%te',)


[<Adult(age='39',workclass=' State-gov', fnlwgt='77516', education=' Bachelors', education_num='13', martial_status=' Never-married', occupation= Adm-clerical, relationship= Not-in-family, race= White, sex=' Male', capital_gain='2174', capital_loss='0', hours_per_week='40', native_country=' United-States', prediction=' <=50K')>,
 <Adult(age='31',workclass=' Private', fnlwgt='45781', education='PhD', education_num='14', martial_status=' Never-married', occupation= Prof-specialty, relationship= Not-in-family, race= White, sex=' Female', capital_gain='14084', capital_loss='0', hours_per_week='50', native_country=' United-States', prediction=' >50K')>,
 <Adult(age='42',workclass=' Private', fnlwgt='159449', education=' Bachelors', education_num='13', martial_status=' Married-civ-spouse', occupation= Exec-managerial, relationship= Husband, race= White, sex=' Male', capital_gain='5178', capital_loss='0', hours_per_week='40', native_country=' United-States', prediction=' >50K')>,
 <Adult(age=

### 2.5 : Write two function queries

In [280]:
from sqlalchemy import func
stmt5 = session.query(func.count(Adult.workclass), Adult.workclass).group_by(Adult.workclass).all()
stmt5

2019-03-21 10:57:13,460 INFO sqlalchemy.engine.base.Engine SELECT count(adult.workclass) AS count_1, adult.workclass AS adult_workclass 
FROM adult GROUP BY adult.workclass


INFO:sqlalchemy.engine.base.Engine:SELECT count(adult.workclass) AS count_1, adult.workclass AS adult_workclass 
FROM adult GROUP BY adult.workclass


2019-03-21 10:57:13,463 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


[(5, ' Private'), (1, ' State-gov')]

In [282]:
stmt6 = session.query(func.sum(Adult.capital_gain)).all()
stmt6

2019-03-21 10:59:46,335 INFO sqlalchemy.engine.base.Engine SELECT sum(adult.capital_gain) AS sum_1 
FROM adult


INFO:sqlalchemy.engine.base.Engine:SELECT sum(adult.capital_gain) AS sum_1 
FROM adult


2019-03-21 10:59:46,339 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


[(43200)]